In [1]:
# Your API key (on the e-mail)
# Andre: 4eadeb76-03d6-40cc-b858-27b0da441ac5
# Pedro: 175accd9-4b3d-42a7-998e-85600fa3d1f4
# Pierre: 7a1ed46e-6371-445e-a383-cc6ccf0525fd
API_KEY = "4eadeb76-03d6-40cc-b858-27b0da441ac5" 

# Your sciper number. 
# Andre: 376762
# Pedro: 374339
# Pierre: 385070
SCIPER = 376762


# To actually rank the preference pairs, or to not save anything (so we can e.g. adjust the prompt we pass to ChatGPT)
playground = False

import gpt_wrapper
gpt_wrapper.api_base = "http://mnlp-backend-938795011.eu-central-1.elb.amazonaws.com"
gpt_wrapper.api_key = API_KEY

from gpt_wrapper.chat import Chat

In [20]:
import json
import random

# Open json file
questions = json.load(open(f"data/{SCIPER}_base.json"))
progress_questions = json.load(open(f"data/{SCIPER}.json"))

# Save a backup just in case
with open(f"data/{SCIPER}_bak.json", "w") as f:
    json.dump(progress_questions, f)

num_questions = len(questions)
num_progress = len(progress_questions)

print(f"Howdy! :D")
print(f"Missing {num_questions - num_progress} questions!")



Howdy! :D
Missing 0 questions!


In [3]:
print(questions[0])

{'course_id': 0, 'question_id': 5738, 'question_body': 'The Murphy Law states that if there is a single security hole in an exposed cryptosystem, then\\dots', 'question_options': ['hope for the best', 'nobody will look for it', 'nobody will find it', 'someone will ultimately find it']}


In [4]:
chat1 = Chat.create("Test Chat")


In [5]:
print(chat1.ask("What did I ask?"))

You asked, "What did I ask?"


In [6]:
# Create the template for the prompt that will be passed to ChatGPT
# Leave the <question> tag where the question should be inserted, as it will later be replaced by the actual question
# TODO: make two templates for open-ended and multi-choice questions
# TODO: read the suggested papers
# TODO: play with the templates to find a good one
template_open_A = """
From now on, act as a student at École Polytechnique Fédérale de Lausanne (EPFL) doing a test. You are about to answer a question. Pay close attention to the theme of the question (Physics, thermodynamics, ...) and think step-by-step when answering the question. 
Please explain the reasoning and assumptions behind your answer. If possible, use specific examples or evidence to support your answer of why it is correct. Moreover, only and only when strictly related to the question and not obvious, address any potential ambiguities or limitations in your answer, in order to provide a more complete and accurate response.<latex>


The question is:
<question>
"""

template_mcq_A = """
From now on, act as a student at École Polytechnique Fédérale de Lausanne (EPFL) doing a test. You are about to answer a question. Pay close attention to the theme of the question (Physics, thermodynamics, ...) and think step-by-step when answering the question.<latex>


The question is:
<question>
"""

template_open_B = """
Q: <question>



A: Let's think step-by-step. 
"""

template_mcq_B = """
Question:

<question>



Answer: Let's think step-by-step. 
"""

# template_open = """
#     From now on, act as a student at École polytechnique fédérale de Lausanne (EPFL)  doing a test. You are about to answer a question. Pay close attention to the theme of the question (Physics, thermodynamics, ...) and think step-by-step when answering the question. 
# Please explain the reasoning and assumptions behind your answer. If possible, use specific examples or evidence to support your answer of why it is correct. Moreover, please address any potential ambiguities or limitations in your answer,  only if the limitations are not obvious and strictly related to the question, in order to provide a more complete and accurate response.

# The question is:
# <question>
# """
# template_open = """
# Let's think step-by-step. Please answer the following question:
# <question>
# """

# template_mcq = """
# Let's think step-by-step. Please answer the following question:
# <question>
# """

# Set the specific question you want to ask
if playground:
    num_progress = 93

In [7]:
import latex_commands
from IPython.display import clear_output

In [19]:

while num_progress < num_questions:

    if playground:
        chatA = Chat.create("Playground chat A " + str(random.randint(0, 10000000000)))
        chatB = Chat.create("Playground chat B " + str(random.randint(0, 10000000000)))
    else:
        chatA = Chat.create("Chat A for question " + str(num_progress) + " " + str(random.randint(0, 10000000000)))
        chatB = Chat.create("Chat B for question " + str(num_progress) + " " + str(random.randint(0, 10000000000)))
    
    template_A = template_open_A
    template_B = template_open_B
    
    questionData = questions[num_progress]
    
    question = questionData["question_body"]
    
    if questionData["question_options"] != None:
        # if it's a multiple choice question, change the template
        template_A = template_mcq_A
        template_B = template_mcq_B
        
        options = questionData["question_options"]
        for i, option in enumerate(options):
            question += f"\n{chr(ord('A') + i)}: {option}"
    
    toAsk_A = template_A.replace("<question>", question)
    toAsk_B = template_B.replace("<question>", question)
    if any([command in question for command in latex_commands.commands]):
        toAsk_A = toAsk_A.replace("<latex>", " The question contains LaTeX commands. Please take this into account when answering.") 
    else:
        toAsk_A = toAsk_A.replace("<latex>", "")

    print("Asked A:")
    print(toAsk_A)
    print("--------------")
    print("Asked B:")
    print(toAsk_B)
   
    # model_args = {
    #     "temperature": 0.5,
    #     "top_p": 0.95
    # }
    
    responseA = chatA.ask(toAsk_A)
    # responseA = ""
    responseB = chatB.ask(toAsk_B)
    A = str(responseA)
    B = str(responseB)
    
    # Depending on the passed template, change this so that it extracts the answer from the response accordingly
    # A_start = response_str.find("<A_START>") + len("<A_START>")
    # A_end = response_str.find("<A_END>")
    # B_start = response_str.find("<B_START>") + len("<B_START>")
    # B_end = response_str.find("<B_END>")
    
    # A = response_str[A_start:A_end].split("A: ")[1].strip()
    # B = response_str[B_start:B_end].split("B: ")[1].strip()

    
    print("--------------")
    print("Answer A:")
    print(A)
    print("--------------")
    print("Answer B:")
    print(B)
    repeat = -1
    while repeat != "n" :
        repeat = input("Repeat? (y/n): ") 
        if repeat == "y":
            if playground:
                chatA = Chat.create("Playground chat A " + str(random.randint(0, 10000000000)))
                chatB = Chat.create("Playground chat B " + str(random.randint(0, 10000000000)))
            else:
                chatA = Chat.create("Chat A for question " + str(num_progress) + " " + str(random.randint(0, 10000000000)))
                chatB = Chat.create("Chat B for question " + str(num_progress) + " " + str(random.randint(0, 10000000000)))
            
            responseA = chatA.ask(toAsk_A)
            responseB = chatB.ask(toAsk_B)
            A = str(responseA)
            B = str(responseB)
            print("---AFTER REP-----------")
            print("Answer A:")
            print(A)
            print("--------------")
            print("Answer B:")
            print(B)
        
    overall, correctness, relevance, clarity, completeness, other = -1, -1, -1, -1, -1, -1
    while overall not in ["A", "B", "AB", "N"]:
        overall = input("Overall ranking (A, B, AB, N for None): ")
    while correctness not in ["A", "B", "AB", "N"]:
        correctness = input("Correctness ranking (A, B, AB, N): ")
    while relevance not in ["A", "B", "AB", "N"]:
        relevance = input("Relevance ranking (A, B, AB, N): ")
    while clarity not in ["A", "B", "AB", "N"]:
        clarity = input("Clarity ranking (A, B, AB, N): ")
    while completeness not in ["A", "B", "AB", "N"]:
        completeness = input("Completeness ranking (A, B, AB, N): ")
    
    other = input("Other ranking (None or `<criteria1>: <A, B, AB>;<criteria2>: <A, B, AB>;...`): ")
    
    if overall == "N":
        overall = "None"
    if correctness == "N":
        correctness = "None"
    if relevance == "N":
        relevance = "None"
    if clarity == "N":
        clarity = "None"
    if completeness == "N":
        completeness = "None"
    if other == "N" or other == "":
        other = "None"
    
    
    progress_questions += [{
        "course_id": questionData["course_id"],
        "question_id": questionData["question_id"],
        "question": questionData["question_body"],
        
        "A_chat_id": responseA.chat_id,
        "B_chat_id": responseB.chat_id,
        
        "A": A, 
        "B": B,
        
        "ranking_criteria": {
            "overall": overall,  
            "correctness": correctness,  
            "relevance": relevance,  
            "clarity": clarity,  
            "completeness": completeness,  
            "other": other,  
        }
    }] 
    
    # Saves first to a backup file, then to the actual file
    if not playground:
        with open(f"data/{SCIPER}_bak.json", "w") as f:
            json.dump(progress_questions, f)
            
        with open(f"data/{SCIPER}.json", "w") as f:
            json.dump(progress_questions, f)

        num_progress += 1
        
    clear_output(False)
